In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import udf
import os
from pyspark.sql.functions import col 
from pyspark.sql import DataFrame
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_date, date_format, concat_ws, lit
from pyspark.sql.functions import when
from pyspark.sql.types import StringType
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import split, col
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower
from pyspark.sql.functions import col

# warehouse_location
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = builder.getOrCreate()

In [2]:
# Diretórios no HDFS que você deseja ler
hdfs_path_stopandsearch = "hdfs://hdfs-nn:9000/datasets/bronze/crimes_dataset/stopandsearch"
hdfs_path_stopandsearch1 = "hdfs://hdfs-nn:9000/datasets/bronze/crimes_dataset/stopandsearch1"

# Ler os dados de cada diretório em DataFrames separados
df_stopandsearch = spark.read.option("delimiter", ",").option("header", "true").csv(hdfs_path_stopandsearch)
df_stopandsearch1 = spark.read.option("delimiter", ",").option("header", "true").csv(hdfs_path_stopandsearch1)

# Combina os DataFrames usando a função union
stopandsearch_df = df_stopandsearch.union(df_stopandsearch1)


+--------------------+--------------------+----------------------------+------------------+---------+---------+------+---------+----------------------+-------------------------+--------------------+----------------+--------------------+----------------------------------+----------------------------------------+
|                Type|                Date|Part of a policing operation|Policing operation| Latitude|Longitude|Gender|Age range|Self-defined ethnicity|Officer-defined ethnicity|         Legislation|Object of search|             Outcome|Outcome linked to object of search|Removal of more than just outer clothing|
+--------------------+--------------------+----------------------------+------------------+---------+---------+------+---------+----------------------+-------------------------+--------------------+----------------+--------------------+----------------------------------+----------------------------------------+
|Person and Vehicl...|2019-07-31T23:00:...|                  

In [4]:
# ----------------------TRANSFORMAÇÃO SILVER STOP_AND_SEARCH-------------------------------
#Renomear a coluna
stopandsearch_df = stopandsearch_df.withColumnRenamed("Part of a policing operation", "STOP Operation")

#Renomear a coluna
stopandsearch_df = stopandsearch_df.withColumnRenamed("Outcome linked to object of search", "Outcome of Searched object")

#Renomear a coluna
stopandsearch_df = stopandsearch_df.withColumnRenamed("Removal of more than just outer clothing", "Clothes removing")

# Dividir a coluna "date" em colunas separadas para ano e mês
stopandsearch_df = stopandsearch_df.withColumn("Year", year(stopandsearch_df["date"]))
stopandsearch_df = stopandsearch_df.withColumn("Month", month(stopandsearch_df["date"]))

# Use a função "when" para criar a nova coluna com categorias
stopandsearch_df = stopandsearch_df.withColumn("Age_category", 
                   when(col("Age range").isin("0-24", "under 10"), "0-24")
                   .when(col("Age range") == "25-34", "25-34")
                   .when(col("Age range") == "over 34", "34+")
                   .when(col("Age range") == "18-24", "0-24")  # Adicionamos esta linha para incluir "18-24"
                   .otherwise("Outro"))

# Converta a coluna "date" em um tipo de data
stopandsearch_df = stopandsearch_df.withColumn("date", to_date("date", "yyyy-MM-dd'T'HH:mm:ssXXX"))


# Função para extrair o nome da cidade do nome do arquivo
def extract_city_name(filename):
    # Obter o nome do arquivo (sem o caminho)
    filename = os.path.basename(filename)
    # Dividir o nome do arquivo usando "-" como delimitador
    parts = filename.split("-")
    if len(parts) >= 3:
        # Pegar o nome da cidade central
        city_name = " ".join(parts[2:-1])
        # Remover "stop and" do nome da cidade
        city_name = city_name.replace("stop and", "").strip()
        return city_name
    else:
        return None

# Registar a função UDF para uso em Spark
extract_city_udf = udf(extract_city_name, StringType())

# Adicionar uma nova coluna "Location" com base no nome do arquivo
stopandsearch_df = stopandsearch_df.withColumn("location", extract_city_udf(input_file_name()))

#Colocar _ em vez de espaços
def rename_columns(dataframe: DataFrame) -> DataFrame:
    for column in dataframe.columns:  
        new_column_name = column.replace(" ", "_")
        dataframe = dataframe.withColumnRenamed(column, new_column_name) 
    return dataframe
stopandsearch_df = rename_columns(stopandsearch_df)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Age range` cannot be resolved. Did you mean one of the following? [`Age_range`, `Age_category`, `Gender`, `Legislation`, `Year`].;
'Project [Type#357, date#377, STOP_Operation#397, Policing_operation#417, Latitude#437, Longitude#457, Gender#477, Age_range#497, Self-defined_ethnicity#517, Officer-defined_ethnicity#537, Legislation#557, Object_of_search#577, Outcome#597, Outcome_of_Searched_object#617, Clothes_removing#637, Year#794, Month#814, CASE WHEN 'Age range IN (0-24,under 10) THEN 0-24 WHEN ('Age range = 25-34) THEN 25-34 WHEN ('Age range = over 34) THEN 34+ WHEN ('Age range = 18-24) THEN 0-24 ELSE Outro END AS Age_category#834, location#717]
+- Project [Type#357, date#377, STOP_Operation#397, Policing_operation#417, Latitude#437, Longitude#457, Gender#477, Age_range#497, Self-defined_ethnicity#517, Officer-defined_ethnicity#537, Legislation#557, Object_of_search#577, Outcome#597, Out

In [6]:
#############GOLD##########################
# Crie uma nova coluna "data" no formato "dd/MM/yyyy"
stopandsearch_df = stopandsearch_df.withColumn("Data", date_format("date", "dd/MM/yyyy"))
stopandsearch_df = stopandsearch_df.withColumn("Data", to_date(stopandsearch_df["data"], "dd/MM/yyyy"))


# Eliminar linhas com as seguintes localizações
locations_to_remove = ['west midlands', 'thames valley', 'south wales', 'cheshire', 'dyfed powys', 'west mercia', 'gwent', 'humberside', 'north wales', 'devon and cornwall']

# Filtre o DataFrame para excluir as linhas com as localizações especificadas
stopandsearch_df = stopandsearch_df[~stopandsearch_df['Location'].isin(locations_to_remove)]

# Janela para gerar IDs sequenciais
w = Window.orderBy("Year")

# Criação das tabelas de dimensões com ID único
dim_data = stopandsearch_df.select(
    "Data",
    "Year",
    "Month"
).distinct().withColumn("id_data", row_number().over(w))

w = Window.orderBy("Latitude", "Longitude")
dim_local = stopandsearch_df.select(
    "Latitude",
    "Longitude",
    "Location"
).distinct().withColumn("id_local", row_number().over(w))

w = Window.orderBy("Gender", "Age_category")
dim_pessoa = stopandsearch_df.select(
    "Gender",
    "Age_category",
    "Self-defined_ethnicity",
    "Officer-defined_ethnicity"
).distinct().withColumn("id_pessoa", row_number().over(w))

w = Window.orderBy("Type")
dim_operation = stopandsearch_df.select(
    "Type",
    "Legislation",
    "Object_of_search",
    "Outcome"
    # Inclua outras colunas necessárias
).distinct().withColumn("id_operation", row_number().over(w))

# Criação da tabela de fatos
factos = stopandsearch_df \
    .join(dim_data, ["Data", "Year", "Month"]) \
    .join(dim_local, ["Latitude", "Longitude", "Location"]) \
    .join(dim_pessoa, ["Gender", "Age_category", "Self-defined_ethnicity", "Officer-defined_ethnicity"]) \
    .join(dim_operation, ["Type", "Legislation", "Object_of_search", "Outcome" ]) \
    .select(
        "id_data", 
        "id_local", 
        "id_pessoa", 
        "id_operation"
        # Adicione outras colunas e métricas conforme necessário
    )



# Salvar a tabela de fatos no formato desejado
# factos.write.format("parquet").save("path_to_save")


+----------+----+-----+-------+
|      Data|Year|Month|id_data|
+----------+----+-----+-------+
|      null|null| null|      1|
|2017-05-26|2017|    5|      2|
|2017-10-17|2017|   10|      3|
|2017-05-16|2017|    5|      4|
|2017-05-31|2017|    5|      5|
|2017-04-30|2017|    4|      6|
|2017-05-29|2017|    5|      7|
|2017-05-05|2017|    5|      8|
|2017-05-10|2017|    5|      9|
|2017-05-02|2017|    5|     10|
|2017-05-06|2017|    5|     11|
|2017-05-23|2017|    5|     12|
|2017-05-17|2017|    5|     13|
|2017-05-27|2017|    5|     14|
|2017-05-24|2017|    5|     15|
|2017-05-30|2017|    5|     16|
|2017-05-11|2017|    5|     17|
|2017-05-03|2017|    5|     18|
|2017-05-21|2017|    5|     19|
|2017-10-03|2017|   10|     20|
+----------+----+-----+-------+
only showing top 20 rows

+--------+---------+-----------------+--------+
|Latitude|Longitude|         Location|id_local|
+--------+---------+-----------------+--------+
|    null|     null|       lancashire|       1|
|    null|    

NameError: name 'show' is not defined

In [7]:
# Para a tabela de dimensão 'Data'
dim_data.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.dim_data")

# Para a tabela de dimensão 'Local'
dim_local.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.dim_local")

# Para a tabela de dimensão 'Pessoa'
dim_pessoa.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.dim_pessoa")

# Para a tabela de dimensão 'Operation'
dim_operation.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.dim_operation")

# Para a tabela de fatos
factos.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.factos")
